## 1. Let's import all the libraries

In [1]:
# https://dpb.bitbucket.io/reloading-modified-code-when-using-the-ipython-interactive-shell.html
%load_ext autoreload
%autoreload 1 # from now on, always reload those modules marked with %aimport before executing any Python code

In [2]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"]    = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import numpy as np
np.random.seed(666) # Diabolic Seed is always better :D

from PIL import Image

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import tensorflow as tf

Using TensorFlow backend.


In [4]:
# https://dpb.bitbucket.io/reloading-modified-code-when-using-the-ipython-interactive-shell.html
# Autoreloading the library whenever it is changed.
%aimport anogan 
from anogan import AnoGAN
from random_names import random_name

In [5]:
flags = tf.app.flags

###############################################################################################################################

# MISC Settings
flags.DEFINE_string("output_dirname", "output", "Target Folder to save images and saved models")

# Input Settings
flags._define_helper("input_shape", (64, 64, 1), "Image Input Shape", list)
flags.DEFINE_integer("batch_size", 64, "The size of batch images [64]")
flags.DEFINE_string("input_dir_training", '../dataset/train/', "Path to the directory containing training data")
flags.DEFINE_string("input_dir_validation", '../dataset/validation/', "Path to the directory containing validation data")

# Latent Shape Definition
flags._define_helper("latent_shape", (100, ), "Size of the latent vector Z", list)

# Network Configuration
flags._define_helper("n_filters", [512, 256, 128, 64], "Convolutions Size for G and D", list)
flags.DEFINE_integer("filter_size", 5, "Kernel Size for the convolutions")
flags.DEFINE_integer("stride", 2, "Kernel Size for the convolutions")
flags.DEFINE_float("gen_lr", 2e-4, "Learning Rate for the Adam Optimizer in G")
flags.DEFINE_float("dis_lr", 2e-4, "Learning Rate for the Adam Optimizer in D")
flags.DEFINE_float("gen_beta1", 0.8, "Beta1 Parameter for the Adam Optimizer in G")
flags.DEFINE_float("dis_beta1", 0.8, "Beta1 Parameter for the Adam Optimizer in D")

# Training Settings
flags.DEFINE_integer("stats_step_interval", 150, "Number of steps between to statistical review of the training")
flags.DEFINE_integer("epochs", 20, "Number of training Epochs to be performed")

###############################################################################################################################

FLAGS = flags.FLAGS

In [6]:
anogan = AnoGAN(
    input_shape         = FLAGS.input_shape, 
    latent_shape        = FLAGS.latent_shape, 
    n_filters           = FLAGS.n_filters, 
    filter_size         = FLAGS.filter_size,
    strides             = FLAGS.stride,
    gen_lr              = FLAGS.gen_lr, 
    gen_beta1           = FLAGS.gen_beta1, 
    dis_lr              = FLAGS.dis_lr,
    dis_beta1           = FLAGS.dis_beta1,
    model_name          = random_name(sep="_"), 
    output_dirname      = FLAGS.output_dirname,
    stats_step_interval=FLAGS.stats_step_interval
)

Model Name: nifty_nightingale


In [7]:
anogan.generator_summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_input (InputLayer) (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 8192)              827392    
_________________________________________________________________
batch_normalization_5 (Batch (None, 8192)              32768     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8192)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 512)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 512)         6554112   
__________

In [8]:
anogan.discriminator_summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discriminator_input (InputLa (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1664      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 128)       0         
__________

In [9]:
anogan.full_GAN_summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_2 (Model)              (None, 64, 64, 1)         11720961  
_________________________________________________________________
model_1 (Model)              (None, 1)                 4315393   
Total params: 16,036,354
Trainable params: 16,016,130
Non-trainable params: 20,224
_________________________________________________________________


In [ ]:
train_datagen = ImageDataGenerator(
    horizontal_flip=True
)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    directory   = FLAGS.input_dir_training,       # this is the target directory
    target_size = FLAGS.input_shape[:-1],         # all images will be resized to 64x64
    batch_size  = FLAGS.batch_size,
    color_mode  = "grayscale",                    # We use a grayscale dataset
    class_mode  = None                            # We do not need to get any label => Everything is healthy
)  

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
    directory   = FLAGS.input_dir_validation,     # this is the target directory
    target_size = FLAGS.input_shape[:-1],         # all images will be resized to 64x64
    batch_size  = FLAGS.batch_size,
    color_mode  = "grayscale",                    # We use a grayscale dataset
    classes     = ["defective", "healthy"],       # {'defective': 0, 'healthy': 1} => Needs to be enforced
    class_mode  = 'binary'                        # We want to have binary labels for validation
)

validation_generator.class_indices

Found 49812 images belonging to 1 classes.
Found 376 images belonging to 2 classes.


{'defective': 0, 'healthy': 1}

In [ ]:
anogan.train_generator(
    X_train_generator = train_generator, 
    X_val_generator   = validation_generator,
    epochs            = FLAGS.epochs
)

Epoch 1 of 20

150/389 [==========>...................] - ETA: 307s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
--------------------------------------------------------------------------------------------------------------
generator (train)                   | 8.939      | 0.257      |         |            |               |        |
discriminator (train)               | 0.793      | 0.811      |         |            |               |        |


300/389 [======================>.......] - ETA: 112s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
--------------------------------------------------------------------------------------------------------------
generator (train)                   | 4.831      | 0.450      |         |            |               |        |
discriminator (train)               | 1.150      | 0.626      |         |            |       

300/389 [======================>.......] - ETA: 108s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
--------------------------------------------------------------------------------------------------------------
generator (train)                   | 0.916      | 0.334      |         |            |               |        |
discriminator (train)               | 1.251      | 0.665      |         |            |               |        |


389/389 [==============================] - 476s   

Epoch processed in 477s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
--------------------------------------------------------------------------------------------------------------
generator (train)                   | 0.932      | 0.320      |         |            |               |        |
discriminator (train)               | 1.257      | 0.666      |         |            

389/389 [==============================] - 476s   

Epoch processed in 477s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
--------------------------------------------------------------------------------------------------------------
generator (train)                   | 0.834      | 0.374      |         |            |               |        |
discriminator (train)               | 1.331      | 0.612      |         |            |               |        |
detection rate (validation)         |            |            | 0.372   | 0.461      | 0.564         | 0.473  |

###################################################################################################################

Epoch 10 of 20

150/389 [==========>...................] - ETA: 291s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
----------------------------------------------------------------

detection rate (validation)         |            |            | 0.048   | 0.346      | 0.910         | 0.489  |

###################################################################################################################

Epoch 14 of 20

150/389 [==========>...................] - ETA: 291s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
--------------------------------------------------------------------------------------------------------------
generator (train)                   | 0.929      | 0.300      |         |            |               |        |
discriminator (train)               | 1.250      | 0.670      |         |            |               |        |


300/389 [======================>.......] - ETA: 109s

network                             | loss       | accuracy   | recall  | precision  | specificity   | NPV    |
-------------------------------------------------------------------------------------

### Plot a few sample from the Generator

In [ ]:
generated_images = anogan.generator.generate_samples(batch_size=100, verbose=0)
            
# arrange them into a grid
img = (np.concatenate([r.reshape(-1, 64)
                       for r in np.split(generated_images, 10)
                       ], axis=-1) * 127.5 + 127.5).astype(np.uint8)

Image.fromarray(img)

### Plot real data for comparison

In [ ]:
generated_images = np.concatenate([train_generator.next(), train_generator.next()])
generated_images = generated_images[:100]
            
# arrange them into a grid
img = (np.concatenate([r.reshape(-1, 64)
                       for r in np.split(generated_images, 10)
                       ], axis=-1)).astype(np.uint8)

Image.fromarray(img)

### Compute Accuracy on Defective Images

In [ ]:
precision, recall, specificity, negative_predictive_value = anogan.detect_defects(validation_generator)

print()
print("Probability of Detecting a Non-Defect             => Recall      (tp / (tp + fn)):", recall)
print("Probability of Correctly Detecting a Non-Defect   => Precision   (tp / (tp + fp)):", precision)
print("Probability of Detecting a Defect                 => Specificity (tn / (tn + fp)):", specificity)
print("Probability of Correctly Detecting a Defect       => NPV         (tn / (tn + fn)):", negative_predictive_value)